# Load Data

In [1]:
import pandas as pd
import numpy as np
import os
import datetime
os.chdir("D:\Dropbox\Data-Analysis\Via\project\data")

In [2]:
df = pd.read_csv('uber-raw-data-janjune-15.csv')
df.head()

,Dispatching_base_num,Pickup_date,Affiliated_base_num,locationID
0,B02617,2015-05-17 09:47:00,B02617,141
1,B02617,2015-05-17 09:47:00,B02617,65
2,B02617,2015-05-17 09:47:00,B02617,100
3,B02617,2015-05-17 09:47:00,B02774,80
4,B02617,2015-05-17 09:47:00,B02617,90


In [3]:
len(df)

14270479

In [4]:
# focus on week 15 data only, 2015-04-06 to 2015-04-12
df['Pickup_date'] = pd.to_datetime(df['Pickup_date'])

In [5]:
df.dtypes

Dispatching_base_num            object
Pickup_date             datetime64[ns]
Affiliated_base_num             object
locationID                       int64
dtype: object

In [6]:
mask = (df['Pickup_date'] > '2015-04-05') & (df['Pickup_date'] <= '2015-04-12')
df2 = df.loc[mask].copy()

In [7]:
df2['Pickup_date'].iloc[0].minute

31

In [13]:
df2['hour'] = df2['Pickup_date'].apply(lambda x: x.hour)
df2['dayofweek'] = df2['Pickup_date'].apply(lambda x: x.dayofweek)
df2.head()

,Dispatching_base_num,Pickup_date,Affiliated_base_num,locationID,hour,dayofweek
8863,B02764,2015-04-07 21:31:00,B02764,244,21,1
8864,B02764,2015-04-07 21:31:00,B00272,161,21,1
8865,B02764,2015-04-07 21:31:00,B02764,37,21,1
8866,B02764,2015-04-07 21:31:00,B02764,231,21,1
8867,B02764,2015-04-07 21:31:00,B02682,234,21,1


In [14]:
# Monday=0, Sunday=6 --> convert to Monday = 1, Sunday = 7
df2['dayofweek']=df2['dayofweek']+1

,Dispatching_base_num,Pickup_date,Affiliated_base_num,locationID,hour,dayofweek
8863,B02764,2015-04-07 21:31:00,B02764,244,21,2
8864,B02764,2015-04-07 21:31:00,B00272,161,21,2
8865,B02764,2015-04-07 21:31:00,B02764,37,21,2
8866,B02764,2015-04-07 21:31:00,B02764,231,21,2
8867,B02764,2015-04-07 21:31:00,B02682,234,21,2


In [15]:
df2.tail()

,Dispatching_base_num,Pickup_date,Affiliated_base_num,locationID,hour,dayofweek
13463090,B02765,2015-04-12,B02765,79,0,7
13463091,B02765,2015-04-12,B02765,114,0,7
13463092,B02765,2015-04-12,B02617,37,0,7
13463093,B02765,2015-04-12,B02765,61,0,7
13463094,B02765,2015-04-12,B02765,113,0,7


In [16]:
len(df2)

493560

In [17]:
zone = pd.read_csv('taxi-zone-lookup.csv')
zone.head()

,LocationID,Borough,Zone
0,1,EWR,Newark Airport
1,2,Queens,Jamaica Bay
2,3,Bronx,Allerton/Pelham Gardens
3,4,Manhattan,Alphabet City
4,5,Staten Island,Arden Heights


In [18]:
df_zone = df2.merge(zone,left_on='locationID',right_on='LocationID',how='left')
df_zone.head()

,Dispatching_base_num,Pickup_date,Affiliated_base_num,locationID,hour,dayofweek,LocationID,Borough,Zone
0,B02764,2015-04-07 21:31:00,B02764,244,21,2,244,Manhattan,Washington Heights South
1,B02764,2015-04-07 21:31:00,B00272,161,21,2,161,Manhattan,Midtown Center
2,B02764,2015-04-07 21:31:00,B02764,37,21,2,37,Brooklyn,Bushwick South
3,B02764,2015-04-07 21:31:00,B02764,231,21,2,231,Manhattan,TriBeCa/Civic Center
4,B02764,2015-04-07 21:31:00,B02682,234,21,2,234,Manhattan,Union Sq


In [19]:
df_zone.drop(['Dispatching_base_num','Affiliated_base_num','locationID','LocationID'],axis=1,inplace=True)           

In [20]:
df_zone.head()   

,Pickup_date,hour,dayofweek,Borough,Zone
0,2015-04-07 21:31:00,21,2,Manhattan,Washington Heights South
1,2015-04-07 21:31:00,21,2,Manhattan,Midtown Center
2,2015-04-07 21:31:00,21,2,Brooklyn,Bushwick South
3,2015-04-07 21:31:00,21,2,Manhattan,TriBeCa/Civic Center
4,2015-04-07 21:31:00,21,2,Manhattan,Union Sq


In [21]:
df_zone.rename(columns = {'Pickup_date':'pickup_datetime','Borough':'pickup_county','Zone':'pickup_neighborhood'},inplace=True)

In [22]:
df_zone.head() 

,pickup_datetime,hour,dayofweek,pickup_county,pickup_neighborhood
0,2015-04-07 21:31:00,21,2,Manhattan,Washington Heights South
1,2015-04-07 21:31:00,21,2,Manhattan,Midtown Center
2,2015-04-07 21:31:00,21,2,Brooklyn,Bushwick South
3,2015-04-07 21:31:00,21,2,Manhattan,TriBeCa/Civic Center
4,2015-04-07 21:31:00,21,2,Manhattan,Union Sq


In [23]:
df_zone.isnull().sum()

pickup_datetime        0
hour                   0
dayofweek              0
pickup_county          0
pickup_neighborhood    0
dtype: int64

In [24]:
df_zone.to_csv('uber_wk15_zone.csv')

In [26]:
def export_dow(df,dow):
    df2 = df[df['dayofweek']==dow].copy()
    return df2

In [29]:
wed = export_dow(df_zone,3)
wed.to_csv('uber_wk15_wed.csv')

In [30]:
sat = export_dow(df_zone,6)
sat.to_csv('uber_wk15_sat.csv')

In [31]:
sun = export_dow(df_zone,7)
sun.to_csv('uber_wk15_sun.csv')